# Tutorial 8: Scheduler - changing hyperparameters during the training

People have been using the same hyperparameters during the training until progressive training emerged. Progressive training is essentially adding a time dimension in hyperparameters and everything can change during the training loop. 

For example:
1. make batch size to be 32 for the 0th epoch, then use 64 on the 5th epoch.
2. train with low resolution image (28x28) for first 3 epochs, then double the resolution (52x52) for another 3 epochs
3. train part of the model for the first 10 epochs, then train another part of the model for 10 more epochs

All of the examples above are examples of hyperparameter change during the training. In FastEstimator, `Scheduler` is used to handle all of these requests. 

## How to use Scheduler

Scheduler can be used in `Pipeline` and `Networks`.  Before using Scheduler, user will need to create a dictionary with key being epoch and value being whatever is used for that epoch. For example:

```python
from fastestimator.util.schedule import Scheduler

mapping = {0: 32, 2:64, 5: 128}
batch_scheduler = Scheduler(epoch_dict=mapping)
```

Then batch_scheduler can be used directly as batch size in `Pipeline`. Please note that the key in the dictionary indicates the epoch of change, therefore, in the example above, when the total training epoch is 8, the batch size for each epoch is:

* epoch 0, batch size 32
* epoch 1, batch size 32
* epoch 2, batch size 64
* epoch 3, batch size 64
* epoch 4, batch size 64
* epoch 5, batch size 128
* epoch 6, batch size 128
* epoch 7, batch size 128

## Scheduler example

In the next example, let's define two image classification models with same architecture(`model1` and `model2`) and train them by the following:

* on epoch 0:  train `model1` with batch size 32, use image resolution 30x30 and Minmax normalization
* on epoch 1:  train `model2` with batch size 64, use image resolution 32x32 and Minmax normalization
* on epoch 2:  train `model1` with batch size 128, use image resolution 30x30 and Rescale normalization(multiply by 1/255)

### Prepare data

In [ ]:
import numpy as np
import tensorflow as tf
import fastestimator as fe

(x_train, y_train), (x_eval, y_eval) = tf.keras.datasets.mnist.load_data()
train_data = {"x": np.expand_dims(x_train, -1), "y": y_train}
eval_data = {"x": np.expand_dims(x_eval, -1), "y": y_eval}
data = {"train": train_data, "eval": eval_data}

### Pipeline

In [ ]:
from fastestimator.util.schedule import Scheduler
from fastestimator.pipeline.processing import Minmax, Resize, Scale

batchsize_scheduler = Scheduler({0:32, 1:64, 2:128})

resize_scheduler = Scheduler({0: Resize(inputs="x", size=(30, 30), outputs="x"),
                              1: Resize(inputs="x", size=(32, 32), outputs="x"),
                              2: Resize(inputs="x", size=(30, 30), outputs="x")})

normalize_scheduler = Scheduler({0: Minmax(inputs="x", outputs="x"),
                                 2: Scale(inputs="x", scalar=1.0/255, outputs="x")})

pipeline = fe.Pipeline(batch_size=batchsize_scheduler, 
                       data=data, 
                       ops=[resize_scheduler, normalize_scheduler])

### Network

In [ ]:
from fastestimator.network.loss import SparseCategoricalCrossentropy
from fastestimator.network.model import FEModel, ModelOp
from fastestimator.architecture import LeNet

model1 = FEModel(model_def=lambda: LeNet(input_shape=(30,30,1)), model_name="model1", optimizer="adam")
model2 = FEModel(model_def=lambda: LeNet(input_shape=(32,32,1)), model_name="model2", optimizer="adam")

model_scheduler = Scheduler({0: ModelOp(inputs="x", model=model1, outputs="y_pred"),
                             1: ModelOp(inputs="x", model=model2, outputs="y_pred"),
                             2: ModelOp(inputs="x", model=model1, outputs="y_pred")})

network = fe.Network(ops=[model_scheduler, SparseCategoricalCrossentropy(inputs=("y", "y_pred"))])

### Estimator

In [ ]:
estimator = fe.Estimator(network=network, pipeline=pipeline, epochs=3)

In [ ]:
estimator.fit()